# EDA

In [2]:
# Import librairies

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import os

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import  OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import cross_val_score, GridSearchCV

path = os.getcwd() + '/'


In [3]:
# Import dataset

df = pd.read_csv(path + "Walmart_Store_sales.csv")

In [4]:
# Get basic statistical infomration on dataset

print("Number of rows : {}".format(df.shape[0]))
print()

print("Display of dataset: ")
display(df.head())
print()

print("Basics statistics: ")
data_desc = df.describe(include='all')
display(data_desc)
print()

print("Percentage of missing values: ")
display(100*df.isnull().sum()/df.shape[0])

Number of rows : 150

Display of dataset: 


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,18-02-2011,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,25-03-2011,1807545.43,0.0,42.38,3.435,128.616064,7.470
2,17.0,27-07-2012,NaN,0.0,NaN,NaN,130.719581,5.936
3,11.0,NaN,1244390.03,0.0,84.57,NaN,214.556497,7.346
4,6.0,28-05-2010,1644470.66,0.0,78.89,2.759,212.412888,7.092



Basics statistics: 


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,150.000000,132,1.360000e+02,138.000000,132.000000,136.000000,138.000000,135.000000
unique,NaN,85,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,19-10-2012,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN
mean,9.866667,NaN,1.249536e+06,0.079710,61.398106,3.320853,179.898509,7.598430
std,6.231191,NaN,6.474630e+05,0.271831,18.378901,0.478149,40.274956,1.577173
min,1.000000,NaN,2.689290e+05,0.000000,18.790000,2.514000,126.111903,5.143000
25%,4.000000,NaN,6.050757e+05,0.000000,45.587500,2.852250,131.970831,6.597500
50%,9.000000,NaN,1.261424e+06,0.000000,62.985000,3.451000,197.908893,7.470000
75%,15.750000,NaN,1.806386e+06,0.000000,76.345000,3.706250,214.934616,8.150000



Percentage of missing values: 


Store            0.000000
Date            12.000000
Weekly_Sales     9.333333
Holiday_Flag     8.000000
Temperature     12.000000
Fuel_Price       9.333333
CPI              8.000000
Unemployment    10.000000
dtype: float64

Elements which stand out:

    1. All columns except "Store" have missing values.
    
    2. Difference between mean and median (understood as the 50% interquartile range) are small on each columns.
    
    3. All values are numeric except date which needs transformation

In [6]:
# Format date column
df["Date"] = pd.to_datetime(df['Date'], dayfirst=True)

In [8]:
# mask missing dates & weekly sales
mask = (df["Weekly_Sales"].notna() & 
        df["Date"].notna())


df = df.loc[mask,:]


In [10]:
# Delcare function to remove outliers by stating dataframe, column and number of standard deviation
def remove_outliers(df,column,n_std):
    mean = df[column].mean()
    sd = df[column].std()
        
    df = df[(df[column] >= mean+(n_std*sd))]
    df = df[(df[column] >= mean-(n_std*sd))]
        
    return df

In [11]:
# remove outliers
remove_outliers(df, "Temperature", 3)
remove_outliers(df, "Fuel_Price", 3)
remove_outliers(df, "CPI", 3)
remove_outliers(df, "Unemployment", 3)

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
30,12.0,2011-05-06,1021154.48,0.0,68.40,4.193,129.044433,13.736
62,12.0,2010-12-17,1295605.35,0.0,52.77,3.236,126.879484,14.313
68,12.0,2011-08-12,955913.68,0.0,91.04,3.701,129.201581,13.503
93,12.0,2011-05-27,964332.51,0.0,NaN,4.087,129.062355,13.736
135,12.0,2010-09-10,903119.03,1.0,83.63,3.044,126.114581,14.180


In [12]:
# Convert relevant columns to datetime index
df['Year'] = pd.DatetimeIndex(df['Date']).year
df['Month'] = pd.DatetimeIndex(df['Date']).month
df['Day'] = pd.DatetimeIndex(df['Date']).day
df['Day_of_week'] = pd.DatetimeIndex(df['Date']).weekday
df = df.drop("Date", axis=1)

In [13]:
# Visualize pairwise dependencies
fig = px.scatter_matrix(df)
fig.update_layout(
        title = go.layout.Title(text = "Bivariate analysis", x = 0.5), showlegend = False, 
            autosize=False, height=1000, width = 1000)
fig.show()

# Preprocessing

In [14]:
# Declare target and features as X and Y
target_name = "Weekly_Sales"
features = df.drop(target_name, axis = 1)


X = features
Y = df[target_name]

In [15]:
# Train/test split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [16]:
# Automatically detect names of numeric/categorical columns
numeric_features = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Year', 'Month', 'Day', 'Day_of_week']
categorical_features = ['Store', 'Holiday_Flag']

In [17]:
# Create pipeline for numeric features
# Names of numeric columns in X_train/X_test
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')), # missing values will be replaced by columns' median
    ('scaler', StandardScaler())
])

In [18]:
# Create pipeline for categorical features
# Names of categorical columns in X_train/X_test
categorical_transformer = Pipeline(
    steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')), # missing values will be replaced by most frequent value
    ('encoder', OneHotEncoder(drop='first')) # first column will be dropped to avoid creating correlations between features
    ])

In [19]:
# Declare preprocessor using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [20]:
# Fit and transform target and features
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

/Users/ymilo/Library/Python/3.8/lib/python/site-packages/sklearn/impute/_base.py:54: FutureWarning:

Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.



# Create and fit model

In [22]:
# Delcare isntance of Linear Regression model
regressor = LinearRegression()

In [23]:
# Train model
print("Train model...")
regressor.fit(X_train, Y_train)
print("...Done.")

Train model...
...Done.


In [25]:
# Performance assessment
# Predictions on training set
print("Predictions on training set...")
Y_train_pred = regressor.predict(X_train)
print("...Done.")
print(Y_train_pred)
print()

Predictions on training set...
...Done.
[ 458010.66615148 2053347.80855057 1929211.47016122  927661.50287212
  556996.66876509  484064.60580178 2105880.88436964  865673.41368794
  379169.7429284  2263830.21101828 2027380.42191327  945154.44447185
 2525783.32310591 1145354.04382678 1457777.82946379 1757242.51
  431454.26044989  875186.05709111 1443448.88611382 1726212.18405704
 2101754.98230145 1418533.7095965   915693.81586049 1525498.30205937
 1006799.77016612  563689.28492874  355682.56266047 1470588.12039206
 1906103.31777096  491607.80874358 2043164.75941008  419819.14199787
  671743.68235464  632865.45148228 1212685.39578116 1921490.17635563
 1499898.34843849  221319.23363078 2045970.06259322 1234699.66027568
 1479255.57749642 1991165.18724555 1564043.2971838   597259.71283863
 1403555.32668624 2457382.34616756  504486.33690541  767161.34764421
 2006430.89058456  448546.86029601  473073.35778044  633683.24810155
  608795.04516351  457772.43929339  523434.80190367 2197374.7937241
 

In [26]:
# Predictions on test set
print("Predictions on test set...")
Y_test_pred = regressor.predict(X_test)
print("...Done.")
print(Y_test_pred)
print()

Predictions on test set...
...Done.
[2168790.21376094 1064714.76439754 2209992.35506412 1242981.65451307
 1985474.24039819  518331.43761397  795906.69405403 1463867.09935462
 2075509.74480472 2007878.42359254  992930.40456631  423407.34695908
 1531842.57046463 2092483.54354465 1309597.51844579 1527519.63434388
  983042.58219149 2200699.68939935  897492.55586591  108008.05490494
 1622921.4369144  1962151.85671882  165476.52683765 1635714.99953292]



In [27]:
# Print R^2 scores
print("R2 score on training set : ", r2_score(Y_train, Y_train_pred))
print("R2 score on test set : ", r2_score(Y_test, Y_test_pred))

R2 score on training set :  0.9795181863004037
R2 score on test set :  0.8979639010132561


# Improve results

In [28]:
# Function to determine the importance of each coefficient in the Regressor
column_names = []
for name, pipeline, features_list in preprocessor.transformers_: # loop over pipelines
    if name == 'num': # if pipeline is for numeric variables
        features = features_list # just get the names of columns to which it has been applied
    else: # if pipeline is for categorical variables
        features = pipeline.named_steps['encoder'].get_feature_names_out() # get output columns names from OneHotEncoder
    column_names.extend(features) # concatenate features names
        
print("Names of columns corresponding to each coefficient: ", column_names)

Names of columns corresponding to each coefficient:  ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Year', 'Month', 'Day', 'Day_of_week', 'x0_2.0', 'x0_3.0', 'x0_4.0', 'x0_5.0', 'x0_6.0', 'x0_7.0', 'x0_8.0', 'x0_9.0', 'x0_10.0', 'x0_11.0', 'x0_12.0', 'x0_13.0', 'x0_14.0', 'x0_15.0', 'x0_16.0', 'x0_17.0', 'x0_18.0', 'x0_19.0', 'x0_20.0', 'x1_1.0']


In [29]:
# Store coefs in a pandas DataFrame
coefs = pd.DataFrame(index = column_names, data = regressor.coef_.transpose(), columns=["coefficients"])
coefs

,coefficients
Temperature,-7.306593e+04
Fuel_Price,-3.655472e+04
CPI,6.574363e+04
Unemployment,-7.756223e+04
Year,-6.367378e+03
Month,7.929570e+04
Day,-3.594910e+04
Day_of_week,-8.731149e-10
x0_2.0,4.510955e+05
x0_3.0,-1.188820e+06


In [30]:
# Compute abs() and sort values
feature_importance = abs(coefs).sort_values(by = 'coefficients')
feature_importance

,coefficients
Day_of_week,8.731149e-10
Year,6.367378e+03
x0_19.0,9.045818e+03
Day,3.594910e+04
Fuel_Price,3.655472e+04
x0_6.0,3.902292e+04
CPI,6.574363e+04
x1_1.0,7.098072e+04
Temperature,7.306593e+04
Unemployment,7.756223e+04


In [31]:
# Plot coefficients
fig = px.bar(feature_importance, orientation = 'h')
fig.update_layout(showlegend = False, 
                  margin = {'l': 120} # to avoid cropping of column names
                 )
fig.show()

In [33]:
# Attempte to acheive better results by using a Ridge regularization model to improve R2 score

print("5-fold cross-validation...")
regressor = Ridge()
scores = cross_val_score(regressor, X_train, Y_train, cv=5)
print('The cross-validated R2-score is : ', scores.mean())
print('The standard deviation is : ', scores.std())

5-fold cross-validation...
The cross-validated R2-score is :  0.865290127531799
The standard deviation is :  0.055081046141942196


In [34]:
# Attempte to acheive better results by using a Lasso regularization model to improve R2 score

print("5-fold cross-validation...")
regressor = Lasso()
scores = cross_val_score(regressor, X_train, Y_train, cv=5)
print('The cross-validated R2-score is : ', scores.mean())
print('The standard deviation is : ', scores.std())

5-fold cross-validation...
The cross-validated R2-score is :  0.9555464724622755
The standard deviation is :  0.010057600746183275
